In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/draft-guide-by-sam-vecenie/draft_guide_data.csv


In [2]:
import pandas as pd
import numpy as np
import time

## Read Data

In [3]:
df = pd.read_csv('/kaggle/input/draft-guide-by-sam-vecenie/draft_guide_data.csv')
print(df.shape)
df.head(1)

(288, 12)


RANK       PLAYER      SCHOOL/TEAM POS  AGE   HT WING  TIER  \
0     1  LaMelo Ball  Illawarra Hawks   G   19  6-6  6-9     5   

                                           STRENGTHS  \
0  Everything starts with Ball’s elite-level feel...   

                                          WEAKNESSES  \
0  The defense isn’t a sure thing, though, becaus...   

                                             SUMMARY     TIER_DESCRIP  
0  Ball should enter the NBA as one of the most c...  All-Star Upside

## Preprocessing


In [4]:
import torch
from torch.nn import functional as F
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from datasets import Dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [6]:
MODEL = 'bert-base-uncased'
CONFIG = {
    "weight_decay": 0.01,
    "num_train_epochs": 10.0,
    "load_best_model_at_end": True,
    "metric_for_best_model": 'RMSE',
}

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [8]:
model.classifier = torch.nn.Linear(
    model.classifier.in_features,
    1
)

In [9]:
# Add Strengths and Weaknesses
df['full_text'] = df[['STRENGTHS', 'WEAKNESSES']].apply(lambda x: x[0] + ' ' + x[1], axis=1)
df.head(1)

RANK       PLAYER      SCHOOL/TEAM POS  AGE   HT WING  TIER  \
0     1  LaMelo Ball  Illawarra Hawks   G   19  6-6  6-9     5   

                                           STRENGTHS  \
0  Everything starts with Ball’s elite-level feel...   

                                          WEAKNESSES  \
0  The defense isn’t a sure thing, though, becaus...   

                                             SUMMARY     TIER_DESCRIP  \
0  Ball should enter the NBA as one of the most c...  All-Star Upside   

                                           full_text  
0  Everything starts with Ball’s elite-level feel...

In [10]:
def preprocess_function(examples):
    tokens = tokenizer(examples['full_text'], 
                       truncation=True, 
                       max_length=512)
    if 'TIER' in examples:
        return {**tokens, "labels": [[float(x)] for x in examples['TIER']]}
    else:
        return tokens

In [11]:
dataset = (
    Dataset.from_pandas(df[['full_text', 'TIER']])
    .map(preprocess_function, batched=True)
    .remove_columns(['TIER'])
    .shuffle(seed=42)
    .train_test_split(seed=42)
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
dataset['train'][0]['labels']

[3.0]

## Modeling

In [13]:
# from sklearn.metrics import mean_squared_error

In [14]:
def MCRMSE(y_pred, y_true):
    """mean columnwise root mean squared error"""
    colwise_root_mse = np.sqrt(
        np.power(y_true - y_pred, 2).mean(axis=0)
    )
    mean_colwise_root_mse = colwise_root_mse.mean()
    return mean_colwise_root_mse

In [15]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # replace with regression loss
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = F.mse_loss(outputs.get("logits"), labels)
        return (loss, outputs) if return_outputs else loss
    
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"MCRMSE": MCRMSE(predictions, labels)}

In [16]:
os.environ["WANDB_DISABLED"] = "true"

torch.cuda.empty_cache()

In [17]:
run_name = f"fpell-{int(time.time())}"

print("=" * 50)
print(f"Starting run: {run_name}")
print("=" * 50)

training_args = TrainingArguments(
    run_name=run_name,
    output_dir="./results",
    save_total_limit=5,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    **CONFIG,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Starting run: fpell-1692027430


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


TrainOutput(global_step=270, training_loss=0.6585798969975224, metrics={'train_runtime': 126.3915, 'train_samples_per_second': 17.09, 'train_steps_per_second': 2.136, 'total_flos': 568314776862720.0, 'train_loss': 0.6585798969975224, 'epoch': 10.0})

In [18]:
temp = trainer.predict(dataset['train'])
MCRMSE(temp[0], temp[1])

0.20601946

In [19]:
temp = trainer.predict(dataset['test'])
MCRMSE(temp[0], temp[1])

1.0615518